# Extract the Travel Time for the Locations of Different Properties

## Import Packages

In [26]:
# Web - Scraping and API Requests
import requests
from httpx import AsyncClient, Response
from parsel import Selector
import parsel
import jmespath
import asyncio

# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json
from typing import List
from typing import TypedDict


# Database Connection
from sqlalchemy import create_engine
from sqlalchemy import inspect, text

# File and System Operations
import os
import sys

## Other Setup

In [27]:
# This allows one to reload the custom package without having to install it again
%load_ext autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# this allows one to reload the custom package without having to install it again
%autoreload 1

sys.path.insert(0,'../src/')

# Import the custom package and sub-packages
%aimport rental_utils
%aimport rental_utils.functions
%aimport rental_utils.sql_queries

In [29]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

### Import Custom Packages

In [30]:
from rental_utils import sql_queries as sqlq
from rental_utils import functions as rent

### Finding the correct file directory for the database credentials json with the api key and password


In [ ]:
current_dir = os.path.dirname(os.path.abspath("NB03_Extract_Travel_Time_Data.ipynb"))
sys.path.insert(0,os.path.join(current_dir, '..'))

credentials_file_path = os.path.join(current_dir, '..', "credentials.json")

# open the  credentials file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

## Augment Property Data with Commute Times

### Load in the Property Data

#### Connect to the Database

In [33]:
engine = sqlq.get_sql_engine("../data/properties.db")

#### Load in the Data from the Corresponding Table in the Database

In [34]:
with engine.connect() as connection:
    rightmove_data = pd.read_sql(text(sqlq.GET_PROPERTIES_DATA_SQL_QUERY), connection)
rightmove_data.head()

,id,price_per_bed,predicted_price_per_bed,travel_time,distance,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,50425854,600.0,None,None,None,1,1.0,29,"St. George Wharf, London, SW8",51.486140,-0.125480,Apartment,new,2025-07-01T15:04:28Z,600,weekly,0,0,rent,1,52 sq. m.,/properties/50425854#/?channel=RES_LET,2017-09-07T14:00:35Z,Added today,1 bedroom apartment
1,80209919,246.0,None,None,None,3,1.0,22,"Dunton Road, London, SE1",51.492600,-0.074660,Terraced,new,2025-07-01T15:46:11Z,738,weekly,0,0,rent,0,70 sq. m.,/properties/80209919#/?channel=RES_LET,2019-03-18T14:08:31Z,Added today,3 bedroom terraced house
2,82567577,669.0,None,None,None,1,1.0,26,"Riverlight Quay, London, SW8",51.480250,-0.133930,Apartment,new,2025-07-01T14:57:09Z,669,weekly,0,0,rent,0,47 sq. m.,/properties/82567577#/?channel=RES_LET,2019-06-13T16:25:32Z,Added today,1 bedroom apartment
3,83161844,1275.0,None,None,None,2,2.0,7,"Greyhound Road, London, W6",51.484998,-0.218458,Apartment,new,2025-07-01T15:45:11Z,2550,monthly,0,0,rent,0,90 sq. m.,/properties/83161844#/?channel=RES_LET,2019-07-09T02:03:16Z,Added today,2 bedroom apartment
4,86137431,460.0,None,None,None,2,2.0,22,"The Modern, Embassy Gardens, London, SW11",51.481964,-0.130297,Apartment,new,2025-07-01T17:30:02Z,920,weekly,0,0,rent,0,,/properties/86137431#/?channel=RES_LET,2023-05-17T09:25:04Z,Added today,2 bedroom apartment


In [35]:
len(rightmove_data)

1047

### Extract the Commute Times for A Chosen Location

#### Set Up the Headers

In [36]:
headers = {
    "Content-Type": "application/json",
    "X-Application-Id": credentials["app_id"],
    "X-Api-Key": credentials["api_key"]
}

#### Define a Function that Takes the Property IDs, Latitudes and Longitudes, and Creates a Custome Payload for the TravelTime API

In [37]:
def create_payload(df: pd.DataFrame, search_id: str="1", transportation_type: str = "public_transport") -> dict:
    """
    Creates a payload dictionary for the TravelTime API using property locations from a DataFrame.
    The payload includes an origin (Bank Station) and destination locations (properties), 
    and sets up the search parameters for a one-to-many public transport commute time query.
    """
    # Define origin (Bank Station - a key commuting hub)
    origin = {
        "id": "Origin",
        "coords": {"lat": 51.513, "lng": -0.088}
    }
    # Ensure the 'id' column is of string type for API compatibility
    df["id"] = df["id"].astype(str)

    # Select and rename latitude/longitude columns for API format
    locations = df[["id", "latitude", "longitude"]].rename(
        columns={"latitude": "lat", "longitude": "lng"}
    )

    # Convert DataFrame rows to a list of dicts for each destination
    destinations = locations.to_dict(orient="records")
    destination_locations = [
        {
            "id": d["id"],
            "coords": {"lat": d["lat"], "lng": d["lng"]}
        } for d in destinations
    ]

    # Build the final payload structure for the API request
    payload = {
        "arrival_searches": {
            "one_to_many": [
                {
                    "id": search_id,  # Unique search identifier
                    "departure_location_id": "Origin",  # Start from Bank Station
                    "arrival_location_ids": df["id"].tolist(),  # List of property IDs as destinations
                    "transportation": {"type": transportation_type},  # Mode of transport
                    "travel_time": 10800,  # Max travel time in seconds (3 hours)
                    "arrival_time_period": "weekday_morning",  # Commute time window
                    "properties": ["travel_time", "distance"]  # Data to return
                }
            ]
        },
        "locations": [origin] + destination_locations  # All locations (origin + destinations)
    }

    return payload

In [38]:
payload_1 = create_payload(rightmove_data)
pprint(payload_1)

{'arrival_searches': {'one_to_many': [{'arrival_location_ids': ['50425854',
                                                                '80209919',
                                                                '82567577',
                                                                '83161844',
                                                                '86137431',
                                                                '87155526',
                                                                '87174426',
                                                                '92029685',
                                                                '95664164',
                                                                '96664666',
                                                                '102345332',
                                                                '106849472',
                                                                '109146980',
         

In [39]:
# Make the request
response = requests.post(
    "https://api.traveltimeapp.com/v4/time-filter/fast",
    headers= headers,
    data=json.dumps(payload_1)
)

# Check the response
print(response.status_code)
pprint(response.json())  # or response.text if not JSON

200
{'results': [{'locations': [{'id': '164007872',
                             'properties': {'distance': 0,
                                            'travel_time': 1428}},
                            {'id': '164028101',
                             'properties': {'distance': 0,
                                            'travel_time': 1105}},
                            {'id': '164007611',
                             'properties': {'distance': 0,
                                            'travel_time': 2706}},
                            {'id': '163314209',
                             'properties': {'distance': 0,
                                            'travel_time': 1929}},
                            {'id': '164021903',
                             'properties': {'distance': 0,
                                            'travel_time': 1801}},
                            {'id': '159121802',
                             'properties': {'distance': 0,
                   

In [40]:
response.status_code == 200

True

### Convert the Response into a New Dataframe with info on Travel Times and Distances for each Property

In [41]:
results = response.json()["results"][0]["locations"]

# Convert to DataFrame
df_results = pd.DataFrame([
    {
        "id": loc["id"],
        "distance": loc["properties"]["distance"],
        "travel_time": loc["properties"]["travel_time"]
    }
    for loc in results
])

# inspect the results
df_results.head(3)

,id,distance,travel_time
0,164007872,0,1428
1,164028101,0,1105
2,164007611,0,2706


### Merge Back With all the Other Original Data about Properties

In [42]:
# Conduct the merge
properties_data = df_results.merge(rightmove_data.drop(columns=["travel_time","distance"]), on="id", how="left")
# Check the data
properties_data.head(3)

,id,distance,travel_time,price_per_bed,predicted_price_per_bed,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,164007872,0,1428,1150.0,None,2,1.0,11,"Victory Parade, London",51.491310,0.072915,Apartment,new,2025-07-01T13:14:03Z,2300,monthly,0,0,rent,0,,/properties/164007872#/?channel=RES_LET,2025-07-01T13:08:06Z,Added today,2 bedroom apartment
1,164028101,0,1105,1875.0,None,2,2.0,4,"Artillery Mansions, 75 Victoria Street, Westmi...",51.497513,-0.134748,Apartment,new,2025-07-01T17:21:07Z,3750,monthly,0,0,rent,0,,/properties/164028101#/?channel=RES_LET,2025-07-01T17:15:49Z,Added today,2 bedroom apartment
2,164007611,0,2706,1400.0,None,1,1.0,8,"Eagle Drive, Colindale",51.597440,-0.247562,Flat,new,2025-07-01T13:10:04Z,1400,monthly,0,0,rent,0,,/properties/164007611#/?channel=RES_LET,2025-07-01T13:04:59Z,Added today,1 bedroom flat


### Save the Data the Database Into the Old Table

In [43]:
#### create a temporary table
sqlq.make_table(properties_data[["id", "travel_time", "distance"]], "temp_updates", engine, if_exists="replace")

#### run a single SQL statement to update properties_data using temp_updates (fill in missing data)
with engine.begin() as connection:
    connection.execute(text(sqlq.UPDATE_DIST_AND_TRAVEL_TIME))

### Check if it can be Extracted OK

In [44]:
with engine.connect() as connection:
    properties_data = pd.read_sql(text(sqlq.GET_PROPERTIES_DATA_SQL_QUERY), connection)
properties_data.head()

,id,price_per_bed,predicted_price_per_bed,travel_time,distance,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,50425854,600.0,None,1059,0,1,1.0,29,"St. George Wharf, London, SW8",51.486140,-0.125480,Apartment,new,2025-07-01T15:04:28Z,600,weekly,0,0,rent,1,52 sq. m.,/properties/50425854#/?channel=RES_LET,2017-09-07T14:00:35Z,Added today,1 bedroom apartment
1,80209919,246.0,None,1368,0,3,1.0,22,"Dunton Road, London, SE1",51.492600,-0.074660,Terraced,new,2025-07-01T15:46:11Z,738,weekly,0,0,rent,0,70 sq. m.,/properties/80209919#/?channel=RES_LET,2019-03-18T14:08:31Z,Added today,3 bedroom terraced house
2,82567577,669.0,None,1572,0,1,1.0,26,"Riverlight Quay, London, SW8",51.480250,-0.133930,Apartment,new,2025-07-01T14:57:09Z,669,weekly,0,0,rent,0,47 sq. m.,/properties/82567577#/?channel=RES_LET,2019-06-13T16:25:32Z,Added today,1 bedroom apartment
3,83161844,1275.0,None,2103,0,2,2.0,7,"Greyhound Road, London, W6",51.484998,-0.218458,Apartment,new,2025-07-01T15:45:11Z,2550,monthly,0,0,rent,0,90 sq. m.,/properties/83161844#/?channel=RES_LET,2019-07-09T02:03:16Z,Added today,2 bedroom apartment
4,86137431,460.0,None,1532,0,2,2.0,22,"The Modern, Embassy Gardens, London, SW11",51.481964,-0.130297,Apartment,new,2025-07-01T17:30:02Z,920,weekly,0,0,rent,0,,/properties/86137431#/?channel=RES_LET,2023-05-17T09:25:04Z,Added today,2 bedroom apartment
